In [1]:
import mysql.connector
import json
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
# Load database configuration from a JSON file
with open("config.json", "r") as f:
    config = json.load(f)

In [38]:
# Create a database connection
cnx = mysql.connector.connect(
    host="localhost",  # Update with your host
    user=config["username"],
    password=config["password"],
    database="project"
)
# Create a cursor object to execute SQL queries
cursor = cnx.cursor()


In [26]:
def get_stats_for_post(post_id):
    query1 = f"SELECT Upvotes+Downvotes FROM Post WHERE PostID={post_id};"
    cursor.execute(query1)
    total_votes= cursor.fetchall()
    total_votes=total_votes[0][0]
    query2=f"SELECT COUNT(*) FROM Comment WHERE PostID={post_id}"
    cursor.execute(query2)
    total_comments= cursor.fetchall()
    total_comments=total_comments[0][0]
    query3=f"SELECT Content FROM Comment WHERE PostID={post_id}"
    cursor.execute(query3)
    comments= cursor.fetchall()
    neg=0
    pos=0
    neut=0
    for comment in comments:
        comment=comment[0]
        sentiment_scores = sid.polarity_scores(comment)
        compound_score = sentiment_scores['compound']
        if compound_score<-0.05:
            neg+=1
        elif compound_score>0.05:
            pos+=1
        else:
            neut+=1

    query4=f"SELECT Downvotes FROM Post WHERE PostID={post_id};"
    cursor.execute(query4)
    downvotes=cursor.fetchall()
    downvotes=downvotes[0][0]
    return [total_votes,total_comments,neg,pos,neut,downvotes]


In [27]:
def rank_posts():
    query0="SELECT DISTINCT PostID from Post;"
    cursor.execute(query0)
    all_posts= cursor.fetchall()
    post_to_score={}
    for post_id in all_posts:
        post_id=post_id[0]
        stats=get_stats_for_post(post_id)
        score=stats[0]+stats[1]-stats[2]+stats[3]-0.5*stats[5]
        post_to_score[post_id]=score
    res= post_to_score.copy()
    sorted_res = sorted(res.items(), key=lambda x:x[1], reverse=True)
    sorted_res = dict(sorted_res)
    return list(sorted_res.keys())

In [28]:
res=rank_posts()

In [29]:
res

[3, 1, 2, 4]

In [18]:
def get_ranked_posts(ranked_ids):
    dfs=[]
    for post_id in ranked_ids:
        query = f"SELECT Post.*,Name FROM Post join User using(UserID) WHERE PostID={post_id};"
        cursor.execute(query)
        result = cursor.fetchall()
        posts_df=pd.DataFrame(result,columns=["PostID","Title","Description","CreatedAtDate","Status","Topic","UserID","Upvotes","Downvotes","OrganizationID","Type","PostedBy"])
        dfs.append(posts_df)

    result_df=pd.concat(dfs,axis=0)
    return result_df


In [19]:
df=get_ranked_posts(res)

In [20]:
df

,PostID,Title,Description,CreatedAtDate,Status,Topic,UserID,Upvotes,Downvotes,OrganizationID,Type,PostedBy
0,3,Afforestation act of 2023- Maharashtra Govt,Planning on increasing area under tree foliage...,2023-11-08 08:06:59,in progress,environment,3,4,0,3,project,atharvaam21
0,1,Jharkhand water crisis,70% Dams have dried up in this summer. Request...,2023-11-08 07:57:21,in progress,environment,1,5,0,None,issue,amaan
0,2,Pune Road Crisis in Baner,Traffic on average 2hrs per day due to poor ro...,2023-11-08 08:02:40,in progress,governance,2,2,2,None,issue,thakre
0,4,Maharashtra Temperature Reach High 20s in Peak...,This unexpected rise in temperatures has left ...,2023-11-08 18:41:25,in progress,environment,4,1,1,1,issue,pratyay


# Levenshtein Distance between two strings

In [30]:
from thefuzz import fuzz

In [33]:
# Check the similarity score
name = "Jharkhand"
full_name = "Jharkhand water crisis"

print(f"Similarity score: {fuzz.ratio(name, full_name)}")



Similarity score: 58


In [43]:
def get_best_matching_titles(search_query):
    query="SELECT Title,PostID FROM Post;"
    cursor.execute(query)
    result=cursor.fetchall()
    titles=[]
    for title in result:
        titles.append([title[0],title[1]])
    string_match_dict={}
    for title in titles:
        string_match_dict[title[1]]=fuzz.ratio(title[0], search_query)
    string_match_dict=sorted(string_match_dict.items(), key=lambda x:x[1], reverse=True)
    string_match_dict = dict(string_match_dict)
    return list(string_match_dict.keys())

In [46]:
get_best_matching_titles("Crisis")

[2, 1, 3, 4, 6]